# Implementação de uma classe capaz de abrir os CSVs de treino e teste do SESA Dataset

A classe deverá ser capaz de receber o caminho de um CSV com dados de TREINO, treinar os classificadores e classificar os dados de outros CSVs de TESTE.

A ideia é criar uma função para receber um único CSV de treino para treinar os classificadores, e eles deverão ficar armazenados na classe, afinal, se a cada CSV de teste eu for fazer o treino antes, o processamento vai ficar pesado. Muito melhor fazer o treinamento uma única vez e depois usar os objetos das classes dos classificadores para testar outros CSVs.

Os melhores classificadores de acordo com o trabalho enviado ao Workshop NUVEM da UFABC, usando esse mesmo dataset, foram:

**1) SGD:** Obteve a segunda melhor acurácia e o melhor tempo de processamento.

**2) Bagging:** Apesar de ter ganhado de apenas 3 classificadores no quesito tempo de processamento, obteve a melhor acurácia.

### Abrindo os CSVs de treino e teste

Para testar tudo, vou usar CSVs com dados ficitícios.

In [121]:
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import mode 
from random import randint
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import time
import json

In [103]:
# MONTANDO O CAMINHO PROS CSVS
csvTreino = '/home/dimi/Programming/IC2019/ML/datasets/SESA/SESA_Normalizado/train/treino_normalizado_semPCA.csv'
csvTeste  = '/home/dimi/Programming/IC2019/ML/datasets/SESA/SESA_Normalizado/test/teste_normalizado_semPCA.csv'

# ABRINDO OS CSVS
dataframeTreino = pd.read_csv(csvTreino) 
dataframeTeste  = pd.read_csv(csvTeste) 

### Funções para fazer o TREINAMENTO dos classificadores

Abaixo, vou escrever as funções responsáveis por treinar os classificadores

#### Função para separar o dataframe em X e Y

Preciso de uma função que pegue o dataframe devolva dois arrays: xTrain e yTrain. Ai depois é só usar esses arrays para treinar os classificadores.

ESSA FUNÇÃO TAMBÉM SERÁ USADA NA PARTE DO TESTE

In [3]:
def separarDataframeXeY(dataframe):
    # PRIMEIRO, MONTO O X (POSSO EXCLUIR A PRIMEIRA COLUNA "ARQUIVOS" E A ÚLTIMA "CLASSE")
    x = dataframe[dataframe.columns[1:-1]]
    
    # AGORA, MONTO O Y
    y = dataframe[dataframe.columns[-1]]
    
    # CONVERTO TUDO PRA TUPLA NORMAL 
    x = x.values.tolist()
    y = y.values.tolist()
    
    return x, y

In [ ]:
#xTrain, yTrain = separarDataframeXeY(dataframeTreino)

#### Função para instanciar classificadores

Como eu não sei se vou usar mais classificadores depois, então vou criar um array de classificadores, assim na hora de treinar eu faço um for loop e vou treinando tudo o que tiver dentro.

Essa função recebe apenas o nome das classes a serem instanciadas. Por exemplo, envie ["SGDClassifier(params)", "BaggingClassifier(params)"] para receber um array de objetos dessas classes.

Além disso, para fazer a validação cruzada, vou instânciar **kCrossValidation** vezes o mesmo classificador. Cada um vai ser treinado com dados diferentes, logo, o resultado será diferente.

In [10]:
def instanciarClassificadores(arrayStringsClassesClassificadores, kCrossValidation=10, verbose=False):
    
    arrayObjClassificadores = []
    
    for classe in arrayStringsClassesClassificadores:
        for k in range(kCrossValidation):
            
            if verbose == True:
                print("Instanciando o " + str(k+1) + "º objeto da classe", classe)
            
            arrayObjClassificadores.append(eval(classe))
        
    return arrayObjClassificadores

In [13]:
arrayObjClassificadores = instanciarClassificadores(["SGDClassifier()", "BaggingClassifier()"], 3, True)
arrayObjClassificadores

Instanciando o 1º objeto da classe SGDClassifier()
Instanciando o 2º objeto da classe SGDClassifier()
Instanciando o 3º objeto da classe SGDClassifier()
Instanciando o 1º objeto da classe BaggingClassifier()
Instanciando o 2º objeto da classe BaggingClassifier()
Instanciando o 3º objeto da classe BaggingClassifier()


[SGDClassifier(alpha=0.0001, average=False, class_weight=None,
               early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
               l1_ratio=0.15, learning_rate='optimal', loss='hinge',
               max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
               power_t=0.5, random_state=None, shuffle=True, tol=0.001,
               validation_fraction=0.1, verbose=0, warm_start=False),
 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
               early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
               l1_ratio=0.15, learning_rate='optimal', loss='hinge',
               max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
               power_t=0.5, random_state=None, shuffle=True, tol=0.001,
               validation_fraction=0.1, verbose=0, warm_start=False),
 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
               early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=T

#### Função para criar um array apenas com os nomes dos arquivos

Vou precisar de um array que contenha o nome de cada arquivo de um CSV, sem repetições. Isso será necessário para criar um dataframe contendo apenas linhas que digam respeito às janelas de um único áudio.

In [15]:
def obterNomesArquivos(dataframe):
    return dataframe[dataframe.columns[0]].unique().tolist()

In [17]:
print(obterNomesArquivos(dataframeTeste)[0:5])

['explosion_208.wav', 'explosion_057.wav', 'casual_054.wav', 'explosion_195.wav', 'gunshot_091.wav']


#### Função para segmentar o dataframe de treino

Cada classificador será treinado só com **x%** de dados do dataframe de treino. Assim, será possível fazer o bootstrap na validação cruzada. Essa função vai receber um dataframe, remover as linhas referentes à alguns arquivos e devolver o novo dataframe segmentado.

In [99]:
def segmentarDataframeTreino(dataframeTreino, percentual=0.75):
    
    # PEGO O NOME DOS AUDIOS NO DATAFRAME
    nomesAudios = obterNomesArquivos(dataframeTreino)
    
    # VEJO QUANTOS ARQUIVOS EU VOU TER QUE TIRAR DO DATAFRAME
    qtdArquivosParaRemover = int(round((1 - percentual) * len(nomesAudios)))
    
    # SELECIONANDO OS ARQUIVOS QUE EU VOU REMOVER
    arquivosParaRemover = []
    for i in range(qtdArquivosParaRemover):
        indexSorteado = randint(0, len(nomesAudios) - 1)
        arquivosParaRemover.append(nomesAudios[indexSorteado])
        del nomesAudios[indexSorteado]
        
    # REMOVENDO 
    for arquivoAtualRemover in arquivosParaRemover:
        dataframeTreino = dataframeTreino[dataframeTreino["nomeArquivo"] != arquivoAtualRemover]
    
    # RETORNANDO
    return dataframeTreino

In [112]:
#dataframeSegmentado = segmentarDataframeTreino(dataframeTreino, percentual=0.01)
#dataframeSegmentado

#### Função para treinar os classificadores

Para fazer a validação cruzada, essa função vai receber um dataframe de treino e, para cada classificador, ela deverá excluir um percentual de arquivos do dataframe e só depois treinar o classificador.

In [119]:
def treinarClassificadores(dataframeTreino, arrayObjClassificadores, percentual=0.75, verbose=False):
    
    # PARA CADA CLASSIFICADOR
    for i, objClassificador in enumerate(arrayObjClassificadores):
        
        # EU REMOVO ALGUNS ARQIVOS DO DATAFRAME ALEATORIAMENTE ATE O PERCENTUAL DEFINIDO
        dataframeTreinoSegmentado = segmentarDataframeTreino(dataframeTreino, percentual)
        
        # SEPARO O NOVO DATAFRAME EM X E Y
        xTrain, yTrain = separarDataframeXeY(dataframeTreinoSegmentado)
        
        # FAÇO UM FIT NO CLASSIFICADOR
        tempoInicio = time.time()
        arrayObjClassificadores[i].fit(xTrain, yTrain)
        tempoFim = time.time()
        
        if verbose == True:
            print("Tempo de treinamento do", objClassificador.__class__.__name__, "(segundos):", tempoFim-tempoInicio)
        
    return arrayObjClassificadores

In [116]:
arrayObjClassificadores = treinarClassificadores(dataframeTreino, arrayObjClassificadores, verbose=True)

Tempo de treinamento do SGDClassifier (segundos): 0.43140339851379395
Tempo de treinamento do SGDClassifier (segundos): 0.46740198135375977
Tempo de treinamento do SGDClassifier (segundos): 0.48363518714904785
Tempo de treinamento do BaggingClassifier (segundos): 4.530040979385376
Tempo de treinamento do BaggingClassifier (segundos): 4.631577253341675
Tempo de treinamento do BaggingClassifier (segundos): 4.6139349937438965


#### Função para unir as três funções anteriores

Essa função vai unir as três funções que foram criadas. Ela vai receber o dataframe de treino e um array de strings contendo os classificadores desejados. 

Em primeiro lugar, ela vai usar a função **separarDataframeXeY** para gerar xTrain e yTrain. Depois, vai usar **instanciarClassificadores** para gerar um array com os classificadores desejados. Por fim, vai treiná-los usando **treinarClassificadores()**.

Ela retorna o array de classificadores com todos eles já treinados e prontos para serem usados para predizer dados.

In [120]:
def criarETreinarClassificadores(dataframeTreino, arrayStringsClassesClassificadores, kCrossValidation=10, percentual=0.75, verbose=False):
    
    if verbose == True:
        print("Começando o treinamento dos classificadores")
    
    arrayObjClassificadores = instanciarClassificadores(arrayStringsClassesClassificadores, kCrossValidation, verbose)
    arrayObjClassificadores = treinarClassificadores(dataframeTreino, arrayObjClassificadores, percentual, verbose)
    
    if verbose == True:
        print("Classificadores treinados")
    
    return arrayObjClassificadores

In [124]:
#arrayObjClassificadores = criarETreinarClassificadores(dataframeTreino, ["KNeighborsClassifier()", "SGDClassifier()", "BaggingClassifier()"], kCrossValidation=3, verbose=True)
#arrayObjClassificadores

### Funções para fazer o TESTE e calcular métricas

Agora vamos implementar as funções responsáveis por pegar um CSV, classificá-lo e calcular as métricas. A classificação deverá ser feita baseada na MODA das classificações de cada janela de um determinado áudio.

#### Função para calcular a moda das classificações de um único áudio

Cada janela de um único áudio será classificada. Depois, para dar um veredito para o áudio como um todo, precisamos tirar a moda das classificações.

In [ ]:
def calcularModa(yPredCadaJanela):
    return mode(yPredCadaJanela)[0][0]

In [ ]:
# PARA TESTAR VOU FAZER UM PREDICT USANDO O XTRAIN MESMO, SÓ PRA VER
a = ["casual", "gunshot", "gunshot", "explosion", "siren"]
print(calcularModa(a))

#### Função para para predizer um conjunto de dados xTest

Essa função receberá uma matriz contendo apenas features de um único áudio e um classificador. Ela vai classificar todas as janelas desse áudio e utilizar a função da moda para dar um veredito sobre esse único áudio. 

A matriz já vai ter que vir arrumada para essa função. As funções que lidam com o pandas para deixar tudo bonitinho vão vir depois.

In [ ]:
def predizerUnicoAudio(xTest, classificador):
    
    yPredCadaJanela = classificador.predict(xTest).tolist()
    
#     print("Janelas classificadas como casual:", yPredCadaJanela.count("casual"))
#     print("Janelas classificadas como gunshot:", yPredCadaJanela.count("gunshot"))
#     print("Janelas classificadas como explosion:", yPredCadaJanela.count("explosion"))
#     print("Janelas classificadas como siren:", yPredCadaJanela.count("siren"))
    
    return calcularModa(yPredCadaJanela)

In [ ]:
# predizerUnicoAudio(xTrain, arrayObjClassificadores[1])

#### Função para criar um array apenas com os nomes dos arquivos

Vou precisar de um array que contenha o nome de cada arquivo de um CSV de teste, sem repetições. Isso será necessário para criar um dataframe contendo apenas linhas que digam respeito às janelas de um único áudio.

In [ ]:
def obterNomesArquivos(dataframe):
    return dataframe[dataframe.columns[0]].unique().tolist()

In [ ]:
# obterNomesArquivos(dataframeTeste)

#### Função para criar um dataframe contendo apenas as linhas referentes a um único áudio

In [ ]:
def obterDataframeUnicoAudio(dataframe, arquivo):
    #print("Obtendo dataframe do áudio", arquivo)
    dataframeArquivoSelecionado = dataframe.loc[dataframe[dataframe.columns[0]] == arquivo]
    #print("Primeiro valor do dataframe:", dataframeArquivoSelecionado[dataframeArquivoSelecionado.columns[1]].values.tolist()[0])
    return dataframeArquivoSelecionado

In [ ]:
# dataframeAudioAtual = obterDataframeUnicoAudio(dataframeTeste, 'D_3.wav')
# dataframeAudioAtual

#### Função para predizer um único áudio e retornar as classificações real e predita

Pois bem, já temos a função de predizer um único áudio, mas ela precisa receber ua matriz bonitinha que só contenha features e que não seja do tipo pandas. 

A função proposta agora deve receber o dataframe tipo pandas de um único áudio, separar o que é xTest e o que é yReal, mandar classificar xTest e devolver yReal e yPred.

In [ ]:
def obterYRealYPredUnicoAudio(dataframeUnicoAudio, classificador):
    # PRIMEIRO EU SEPARO O QUE E X E Y
    xTest, arrayYReal = separarDataframeXeY(dataframeUnicoAudio)
    
    # FACO YREAL SER UM VALOR UNICO JA QUE A FUNCAO ACIMA RETORNA UM ARRAY Y
    yReal = arrayYReal[0]
    
    # PREDIZENDO A CLASSE DO AUDIO EM QUESTAO (A FUNCAO ABAIXO JA RETORNA A MODA)
    yPred = predizerUnicoAudio(xTest, classificador)
    
    #print("Classificacao real:", yReal)
    #print("Classificação atribuída:", yPred)
    
    return yReal, yPred

In [ ]:
#yRealAudioAtual, yPredAudioAtual = obterYRealYPredUnicoAudio(dataframeAudioAtual, arrayObjClassificadores[0])

#### Função para classificar um dataframe de teste inteiro

A função abaixo vai receber um dataframe e um classificador. Depois disso ela vai:

1) Obter um array com o nome dos arquivos desse dataframe utilizando a função **obterNomesArquivos**;

2) Para cada arquivo no dataframe, ela vai criar um novo dataframe contendo as janelas apenas desse único áudio, utilizando, para isso, a função **obterDataframeUnicoAudio**;

3) Obter a classificação real e a predita para cada um dos arquivos utilizando a função **obterYRealYPredUnicoAudio**;

4) Com tudo isso, ela vai montar os arrays yTest e yPred;

5) Retornar o classification report do sklearn como um DICT (depois fica bem facil de transformar em JSON) com todas as informações de precisão, recall, acurácia e tal.

A função que vai salvar o classification report, convertendo-o para um JSON será implementada a diante. Lembrando que essa função também vai ficar responsável por colocar o nome do classificador utilizado no classification report.

In [ ]:
def classificarDataframeCompleto(dataframeTeste, classificador, verbose=False):
    
    print("Classificando com o ", classificador.__class__.__name__)
    
    # CRIANDO OS ARRAYS GERAIS
    yRealCadaAudio = []
    yPredCadaAudio = []
    
    # PEGANDO O NOME DOS ARQUIVOS QUE ESTAO NO DATAFRAME
    arrayNomesArquivos = obterNomesArquivos(dataframeTeste)
    totalArquivos = len(arrayNomesArquivos)
    
    # PARA CADA ARQUIVO NO DATAFRAME DE TESTE
    for i, arquivo in enumerate(arrayNomesArquivos):
        
        if verbose == True:
            print("Classificando " + arquivo + ". Arquivo", i+1, "de", totalArquivos, "->", str(100*((i+1)/totalArquivos))+"%")
        
        # EU CRIO UM DATAFRAME CONTENDO APENAS AS LINHAS DO AUDIO ATUAL
        dataframeAudioAtual = obterDataframeUnicoAudio(dataframeTeste, arquivo)
        
        # E OBTENHO, PARA ESSE AUDIO, A CLASSIFICACAO REAL E A PREDITA
        yRealAtual, yPredAtual = obterYRealYPredUnicoAudio(dataframeAudioAtual, classificador)
        
        # COLOCO O RESULTADO NOS ARRAYS GERAIS
        yRealCadaAudio.append(yRealAtual)
        yPredCadaAudio.append(yPredAtual)
        
    # CRIANDO O CLASSIFICATION REPORT. PARA FAZER ELE PARECER UM JSON, TEM QUE COLOCAR output_dict=True
    # ASSIM ELE RETORNA UM DICT QUE DEPOIS VAI SER CONVERTIDO PRA JSON E SER SALVO EM ALGUM LUGAR
    dictRelatorio = classification_report(yRealCadaAudio, yPredCadaAudio, digits=3, output_dict=True)
    
    # COLOCANDO O NOME DO CLASSIFICADOR NO DICIONARIO
    dictRelatorio['classificador'] = classificador.__class__.__name__
    
    return dictRelatorio

#### Função para salvar o classification report

Essa função vai receber um classification report como um DICT, converte-lo para JSON e salvar num diretório. Dentro desse dicionário já vai vir escrito qual foi o classificador utilizado, pois a função **classificarDataframeCompleto** coloca essa informação antes de retornar esse dicionário do relatório de classificação.

In [ ]:
def salvarRelatorioClassificacao(dictRelatorio, diretorioOndeSalvar, nomeDatasetTreino):
    
    nomeJSON = "relatorio_" + nomeDatasetTreino + "_" + str(time.time()) + ".json"
    
    print("Salvando as informações num JSON:", nomeJSON)
    
    with open(diretorioOndeSalvar + nomeJSON, 'w') as json_file:
        json.dump(dictRelatorio, json_file)

In [ ]:
#diretorio         = "/home/dimi/Programming/IC2019/ML/datasets/SESA/"
#nomeDatasetTreino = "testeEscalonado53D"
#salvarRelatorioClassificacao(dictRelatorio, diretorio, nomeDatasetTreino)

#### Função para classificar um dataset com todos os classificadores e gerar os relatorios

Essa é a função principal da parte de treinamento de dataset, pois ela vai receber um dataset e o array de classificadores já treinados e, para cada classificador, ela vai garar um relátorio de classificação.

In [ ]:
def classificarDataframe(dataframeTeste, arrayObjClassificadores, diretorioOndeSalvar, nomeDatasetTreino, verbose=False):
    
    print("Começando a classificar o dataset", nomeDatasetTreino)
    
    # PARA CADA CLASSIFICADOR
    for classificadorAtual in arrayObjClassificadores:
        
        # EU GERO O RELATORIO DE CLASSIFICACAO COMO UM DICIONARIO
        dictRelatorioAtual = classificarDataframeCompleto(dataframeTeste, classificadorAtual, verbose)
        
        # E SALVO AS INFORMACOES COMO JSON
        salvarRelatorioClassificacao(dictRelatorioAtual, diretorioOndeSalvar, nomeDatasetTreino)

# Finalmente criando e testando a classe

In [132]:
#A classe deverá ser capaz de receber o caminho de um CSV com dados de TREINO, treinar os classificadores e classificar os dados de outros CSVs de TESTE.
#A ideia é criar uma função para receber um único CSV de treino para treinar os classificadores, e eles deverão ficar armazenados na classe, afinal, se a cada CSV de teste eu for fazer o treino antes, o processamento vai ficar pesado. Muito melhor fazer o treinamento uma única vez e depois usar os objetos das classes dos classificadores para testar outros CSVs.

# Toda a implementação da classe está muito bem descrita no jupyter "Implementacao de classe para treinamento e classificacao de CSVs"

# Usando a classe
# 1) Crie o caminho para o CSV de TREINAMENTO e abra-o como um pandas dataframe;
# 2) Utilizando o dataframe de treinamento aberto, crie e treine os classificadores desejados;
# 3) Crie o caminho para o CSV de TESTE, setando as váriaveis diretorio e nomeDataset;
# 4) Classifique o dataset de TESTE aberto, especificando onde o JSON do relátorio deve ser salvo
# Os passos 3 e 4 podem ser repetidos caso se deseje classificar vários datasets de TESTE.

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import mode 
from random import randint
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import time
import json

class TreinarEClassificar:

	def separarDataframeXeY(self, dataframe):
		# Função para separar o dataframe em X e Y
		# Preciso de uma função que pegue o dataframe devolva dois arrays: xTrain e yTrain. Ai depois é só usar esses arrays para treinar os classificadores.

		# ESSA FUNÇÃO TAMBÉM SERÁ USADA NA PARTE DO TESTE

		# PRIMEIRO, MONTO O X (POSSO EXCLUIR A PRIMEIRA COLUNA "ARQUIVOS" E A ÚLTIMA "CLASSE")
		x = dataframe[dataframe.columns[1:-1]]
		
		# AGORA, MONTO O Y
		y = dataframe[dataframe.columns[-1]]
		
		# CONVERTO TUDO PRA TUPLA NORMAL 
		x = x.values.tolist()
		y = y.values.tolist()
		
		return x, y

	def instanciarClassificadores(self, arrayStringsClassesClassificadores, kCrossValidation=10, verbose=False):
		# Função para instanciar classificadores
		# Como eu não sei se vou usar mais classificadores depois, então vou criar um array de classificadores, assim na hora de treinar eu faço um for loop e vou treinando tudo o que tiver dentro.

		# Essa função recebe apenas o nome das classes a serem instanciadas. Por exemplo, envie ["SGDClassifier(params)", "BaggingClassifier(params)"] para receber um array de objetos dessas classes.
		
		#Além disso, para fazer a validação cruzada, vou instânciar kCrossValidation vezes o mesmo classificador. Cada um vai ser treinado com dados diferentes, logo, o resultado será diferente.

		arrayObjClassificadores = []
		
		for classe in arrayStringsClassesClassificadores:
			for k in range(kCrossValidation):
				
				if verbose == True:
					print("Instanciando o " + str(k+1) + "º objeto da classe", classe)
				
				arrayObjClassificadores.append(eval(classe))
			
		return arrayObjClassificadores

	def obterNomesArquivos(self, dataframe):
		# Função para criar um array apenas com os nomes dos arquivos
		# Vou precisar de um array que contenha o nome de cada arquivo de um CSV de teste, sem repetições. Isso será necessário para criar um dataframe contendo apenas linhas que digam respeito às janelas de um único áudio.
		return dataframe[dataframe.columns[0]].unique().tolist()

	def segmentarDataframeTreino(self, dataframeTreino, percentual=0.75):
		#Função para segmentar o dataframe de treino
		#Cada classificador será treinado só com x% de dados do dataframe de treino. Assim, será possível fazer o bootstrap na validação cruzada. Essa função vai receber um dataframe, remover as linhas referentes à alguns arquivos e devolver o novo dataframe segmentado.

		# PEGO O NOME DOS AUDIOS NO DATAFRAME
		nomesAudios = self.obterNomesArquivos(dataframeTreino)
		
		# VEJO QUANTOS ARQUIVOS EU VOU TER QUE TIRAR DO DATAFRAME
		qtdArquivosParaRemover = int(round((1 - percentual) * len(nomesAudios)))
		
		# SELECIONANDO OS ARQUIVOS QUE EU VOU REMOVER
		arquivosParaRemover = []
		for i in range(qtdArquivosParaRemover):
			indexSorteado = randint(0, len(nomesAudios) - 1)
			arquivosParaRemover.append(nomesAudios[indexSorteado])
			del nomesAudios[indexSorteado]
			
		# REMOVENDO 
		for arquivoAtualRemover in arquivosParaRemover:
			dataframeTreino = dataframeTreino[dataframeTreino["nomeArquivo"] != arquivoAtualRemover]
		
		# RETORNANDO
		return dataframeTreino

	def treinarClassificadores(self, dataframeTreino, arrayObjClassificadores, percentual=0.75, verbose=False):
		# Função para treinar os classificadores
		#Para fazer a validação cruzada, essa função vai receber um dataframe de treino e, para cada classificador, ela deverá excluir um percentual de arquivos do dataframe e só depois treinar o classificador.
		
		# PARA CADA CLASSIFICADOR
		for i, objClassificador in enumerate(arrayObjClassificadores):
			
			# EU REMOVO ALGUNS ARQIVOS DO DATAFRAME ALEATORIAMENTE ATE O PERCENTUAL DEFINIDO
			dataframeTreinoSegmentado = self.segmentarDataframeTreino(dataframeTreino, percentual)
			
			# SEPARO O NOVO DATAFRAME EM X E Y
			xTrain, yTrain = self.separarDataframeXeY(dataframeTreinoSegmentado)
			
			# FAÇO UM FIT NO CLASSIFICADOR
			tempoInicio = time.time()
			arrayObjClassificadores[i].fit(xTrain, yTrain)
			tempoFim = time.time()
			
			if verbose == True:
				print("Tempo de treinamento do", objClassificador.__class__.__name__, "(segundos):", tempoFim-tempoInicio)
			
		return arrayObjClassificadores

	def criarETreinarClassificadores(self, dataframeTreino, arrayStringsClassesClassificadores, kCrossValidation=10, percentual=0.75, verbose=False):
		# Função para unir as três funções anteriores
		# Essa função vai unir as três funções que foram criadas. Ela vai receber o dataframe de treino e um array de strings contendo os classificadores desejados.
		# Em primeiro lugar, ela vai usar a função separarDataframeXeY para gerar xTrain e yTrain. Depois, vai usar instanciarClassificadores para gerar um array com os classificadores desejados. Por fim, vai treiná-los usando treinarClassificadores().
		# Ela retorna o array de classificadores com todos eles já treinados e prontos para serem usados para predizer dados.
		
		if verbose == True:
			print("Começando o treinamento dos classificadores")
		
		arrayObjClassificadores = instanciarClassificadores(arrayStringsClassesClassificadores, kCrossValidation, verbose)
		arrayObjClassificadores = treinarClassificadores(dataframeTreino, arrayObjClassificadores, percentual, verbose)
		
		if verbose == True:
			print("Classificadores treinados")
		
		return arrayObjClassificadores

	def calcularModa(self, yPredCadaJanela):
		# Função para calcular a moda das classificações de um único áudio
		# Cada janela de um único áudio será classificada. Depois, para dar um veredito para o áudio como um todo, precisamos tirar a moda das classificações.
		return mode(yPredCadaJanela)[0][0]

	def predizerUnicoAudio(self, xTest, classificador):
		# Função para para predizer um conjunto de dados xTest
		# Essa função receberá uma matriz contendo apenas features de um único áudio e um classificador. Ela vai classificar todas as janelas desse áudio e utilizar a função da moda para dar um veredito sobre esse único áudio.
		# A matriz já vai ter que vir arrumada para essa função. As funções que lidam com o pandas para deixar tudo bonitinho vão vir depois.
		yPredCadaJanela = classificador.predict(xTest).tolist()
		return self.calcularModa(yPredCadaJanela)

	def obterDataframeUnicoAudio(self, dataframe, arquivo):
		# Função para criar um dataframe contendo apenas as linhas referentes a um único áudio
		dataframeArquivoSelecionado = dataframe.loc[dataframe[dataframe.columns[0]] == arquivo]
		return dataframeArquivoSelecionado

	def obterYRealYPredUnicoAudio(self, dataframeUnicoAudio, classificador):
		# Função para predizer um único áudio e retornar as classificações real e predita
		# Pois bem, já temos a função de predizer um único áudio, mas ela precisa receber ua matriz bonitinha que só contenha features e que não seja do tipo pandas.
		# A função proposta agora deve receber o dataframe tipo pandas de um único áudio, separar o que é xTest e o que é yReal, mandar classificar xTest e devolver yReal e yPred.

		# PRIMEIRO EU SEPARO O QUE E X E Y
		xTest, arrayYReal = self.separarDataframeXeY(dataframeUnicoAudio)
		
		# FACO YREAL SER UM VALOR UNICO JA QUE A FUNCAO ACIMA RETORNA UM ARRAY Y
		yReal = arrayYReal[0]
		
		# PREDIZENDO A CLASSE DO AUDIO EM QUESTAO (A FUNCAO ABAIXO JA RETORNA A MODA)
		yPred = self.predizerUnicoAudio(xTest, classificador)
		
		return yReal, yPred  

	def classificarDataframeCompleto(self, dataframeTeste, classificador, verbose=False):
		# Função para classificar um dataframe de teste inteiro
		# A função abaixo vai receber um dataframe e um classificador. Depois disso ela vai:
		# 1) Obter um array com o nome dos arquivos desse dataframe utilizando a função obterNomesArquivos;
		# 2) Para cada arquivo no dataframe, ela vai criar um novo dataframe contendo as janelas apenas desse único áudio, utilizando, para isso, a função obterDataframeUnicoAudio;
		# 3) Obter a classificação real e a predita para cada um dos arquivos utilizando a função obterYRealYPredUnicoAudio;
		# 4) Com tudo isso, ela vai montar os arrays yTest e yPred;
		# 5) Retornar o classification report do sklearn como um DICT (depois fica bem facil de transformar em JSON) com todas as informações de precisão, recall, acurácia e tal.
		# A função que vai salvar o classification report, convertendo-o para um JSON será implementada a diante. Lembrando que essa função também vai ficar responsável por colocar o nome do classificador utilizado no classification report.
		
		print("Classificando com o ", classificador.__class__.__name__)
		
		# CRIANDO OS ARRAYS GERAIS
		yRealCadaAudio = []
		yPredCadaAudio = []
		
		# PEGANDO O NOME DOS ARQUIVOS QUE ESTAO NO DATAFRAME
		arrayNomesArquivos = self.obterNomesArquivos(dataframeTeste)
		totalArquivos = len(arrayNomesArquivos)
		
		# PARA CADA ARQUIVO NO DATAFRAME DE TESTE
		for i, arquivo in enumerate(arrayNomesArquivos):
			
			if verbose == True:
				print("Classificando " + arquivo + ". Arquivo", i+1, "de", totalArquivos, "->", str(100*((i+1)/totalArquivos))+"%")
			
			# EU CRIO UM DATAFRAME CONTENDO APENAS AS LINHAS DO AUDIO ATUAL
			dataframeAudioAtual = self.obterDataframeUnicoAudio(dataframeTeste, arquivo)
			
			# E OBTENHO, PARA ESSE AUDIO, A CLASSIFICACAO REAL E A PREDITA
			yRealAtual, yPredAtual = self.obterYRealYPredUnicoAudio(dataframeAudioAtual, classificador)
			
			# COLOCO O RESULTADO NOS ARRAYS GERAIS
			yRealCadaAudio.append(yRealAtual)
			yPredCadaAudio.append(yPredAtual)
			
		# CRIANDO O CLASSIFICATION REPORT. PARA FAZER ELE PARECER UM JSON, TEM QUE COLOCAR output_dict=True
		# ASSIM ELE RETORNA UM DICT QUE DEPOIS VAI SER CONVERTIDO PRA JSON E SER SALVO EM ALGUM LUGAR
		dictRelatorio = classification_report(yRealCadaAudio, yPredCadaAudio, digits=3, output_dict=True)
		
		# COLOCANDO O NOME DO CLASSIFICADOR NO DICIONARIO
		dictRelatorio['classificador'] = classificador.__class__.__name__
		
		return dictRelatorio

	def salvarRelatorioClassificacao(self, dictRelatorio, diretorioOndeSalvar, nomeDatasetTreino):
		# Função para salvar o classification report
		# Essa função vai receber um classification report como um DICT, converte-lo para JSON e salvar num diretório. Dentro desse dicionário já vai vir escrito qual foi o classificador utilizado, pois a função classificarDataframeCompleto coloca essa informação antes de retornar esse dicionário do relatório de classificação.
		nomeJSON = "relatorio_" + nomeDatasetTreino + "_" + str(time.time()) + ".json"
		
		print("Salvando as informações num JSON:", nomeJSON)
		
		with open(diretorioOndeSalvar + nomeJSON, 'w') as json_file:
			json.dump(dictRelatorio, json_file)

	def classificarDataframe(self, dataframeTeste, arrayObjClassificadores, diretorioOndeSalvar, nomeDatasetTreino, verbose=False):
		# Função para classificar um dataset com todos os classificadores e gerar os relatorios
		# Essa é a função principal da parte de treinamento de dataset, pois ela vai receber um dataset e o array de classificadores já treinados e, para cada classificador, ela vai garar um relátorio de classificação.
		print("Começando a classificar o dataset", nomeDatasetTreino)
		
		# PARA CADA CLASSIFICADOR
		for classificadorAtual in arrayObjClassificadores:
			
			# EU GERO O RELATORIO DE CLASSIFICACAO COMO UM DICIONARIO
			dictRelatorioAtual = self.classificarDataframeCompleto(dataframeTeste, classificadorAtual, verbose)

			print("Acurácia:", dictRelatorioAtual["accuracy"])
			
			# E SALVO AS INFORMACOES COMO JSON
			self.salvarRelatorioClassificacao(dictRelatorioAtual, diretorioOndeSalvar, nomeDatasetTreino)

		print("O teste com todos os classificadores foi finalizado")


## Usando a classe

1) Crie o caminho para o CSV de TREINAMENTO e abra-o como um pandas dataframe;

2) Utilizando o dataframe de treinamento aberto, crie e treine os classificadores desejados;

3) Crie o caminho para o CSV de TESTE, setando as váriaveis diretorio e nomeDataset;

4) Classifique o dataset de TESTE aberto, especificando onde o JSON do relátorio deve ser salvo

Os passos 3 e 4 podem ser repetidos caso se deseje classificar vários datasets de TESTE.


In [133]:
objTreinarEClassificar = TreinarEClassificar()

# ABRINDO O CSV DE TREINO COM PANDAS
caminhoCSVTreino = '/home/dimi/Programming/IC2019/ML/datasets/SESA/SESA_Normalizado/train/treino_normalizado_semPCA.csv'
dataframeTreino = pd.read_csv(caminhoCSVTreino) 

# CRIANDO E TREINANDO OS CLASIFICADORES
arrayStringsClassificadores = ["KNeighborsClassifier()"]
arrayObjClassificadores = objTreinarEClassificar.criarETreinarClassificadores(dataframeTreino, arrayStringsClassificadores, kCrossValidation=5, percentual=0.75, verbose=True)

# ABRINDO O CSV QUE EU DESEJO CLASSIFICAR - DATASET DE TESTE 
diretorio         = '/home/dimi/Programming/IC2019/ML/datasets/SESA/SESA_Normalizado/test/'
nomeDatasetTreino = 'teste_normalizado_semPCA'
caminhoCSVTeste   = diretorio + nomeDatasetTreino + '.csv'
dataframeTeste    = pd.read_csv(caminhoCSVTeste)

# CLASSIFICANDO O DATAFRAME DE TESTE
objTreinarEClassificar.classificarDataframe(dataframeTeste, arrayObjClassificadores, diretorio, nomeDatasetTreino, verbose=False)

Começando o treinamento dos classificadores
Instanciando o 1º objeto da classe KNeighborsClassifier()
Instanciando o 2º objeto da classe KNeighborsClassifier()
Instanciando o 3º objeto da classe KNeighborsClassifier()
Instanciando o 4º objeto da classe KNeighborsClassifier()
Instanciando o 5º objeto da classe KNeighborsClassifier()
Tempo de treinamento do KNeighborsClassifier (segundos): 0.03890275955200195
Tempo de treinamento do KNeighborsClassifier (segundos): 0.05088186264038086
Tempo de treinamento do KNeighborsClassifier (segundos): 0.03862953186035156
Tempo de treinamento do KNeighborsClassifier (segundos): 0.03982734680175781
Tempo de treinamento do KNeighborsClassifier (segundos): 0.038634300231933594
Classificadores treinados
Começando a classificar o dataset teste_normalizado_semPCA
Classificando com o  KNeighborsClassifier
Acurácia: 0.8476190476190476
Salvando as informações num JSON: relatorio_teste_normalizado_semPCA_1581552922.1057143.json
Classificando com o  KNeighbors